In [15]:
import os
import time
import warnings
import pickle
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.random_projection import johnson_lindenstrauss_min_dim
from sklearn.random_projection import SparseRandomProjection
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
from sklearn.model_selection import StratifiedShuffleSplit, GridSearchCV
from sklearn.exceptions import UndefinedMetricWarning
import zipfile
import csv

warnings.filterwarnings("ignore", category=UndefinedMetricWarning)

###############################################################################
# read data from file

def extract_zip(zip_file, extract_to):
    """Extracts a zip file to the specified directory."""
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

def getDataPoints(path):
    """Collects content of all .py files within the given directory."""
    dataPointsList = []
    if not os.path.exists(path):
        print(f"Directory does not exist: {path}")
        return dataPointsList

    for root, dirs, files in os.walk(path):
        for dataPointName in files:
            if dataPointName.endswith(".py"):  # Only consider Python files
                file_path = os.path.join(root, dataPointName)
                with open(file_path, encoding="utf-8") as fileIn:
                    dp = fileIn.read().strip()
                    if dp:  # Ensure the document is not empty
                        dataPointsList.append(dp)
                    else:
                        print(f"Empty file: {file_path}")
    
    if len(dataPointsList) == 0:
        print(f"No valid documents found in directory: {path}")
    
    return dataPointsList


def computeResults(testLabels, predictLabels):
    try:
        precision = precision_score(testLabels, predictLabels)
        recall = recall_score(testLabels, predictLabels)
        accuracy = accuracy_score(testLabels, predictLabels)
        f1 = f1_score(testLabels, predictLabels)
        print(f"Precision: {precision}, Recall: {recall}, Accuracy: {accuracy}, F1 Score: {f1}")
    except Exception as e:
        print(f"Error computing metrics: {e}")
        precision = recall = accuracy = f1 = "-"
    return precision, recall, accuracy, f1

###############################################################################
# Vectorization

def vectorization(dataPoints, dim=0, eps=0.3):
    countVec = CountVectorizer()
    Z_full = countVec.fit_transform(dataPoints)
    if eps == 0:
        Z = Z_full
    else:
        if dim <= 0:
            dim = johnson_lindenstrauss_min_dim(Z_full.shape[0], eps=eps)
        srp = SparseRandomProjection(n_components=dim)
        Z = srp.fit_transform(Z_full)
    return Z

###############################################################################
# Classification with Random Forest

def classificationRandomForest(trainData, trainLabels, testData, params):
    # Training
    clf = RandomForestClassifier(
        n_estimators=params.get("n_estimators", 100),
        criterion=params.get("criterion", "gini"),
        max_depth=params.get("max_depth"),
        min_samples_split=params.get("min_samples_split", 2),
        min_samples_leaf=params.get("min_samples_leaf", 1),
        n_jobs=params.get("n_jobs", -1),
        random_state=params.get("random_state", 42)
    )
    
    # Fit the model
    t0 = time.perf_counter()
    clf.fit(trainData, trainLabels)
    trainTime = time.perf_counter() - t0

    # Predict
    t0 = time.perf_counter()
    predictLabels = clf.predict(testData)
    testTime = time.perf_counter() - t0

    return trainTime, testTime, predictLabels

###############################################################################
# Random Forest with Grid Search

def randomForestWithGridSearch(outDir, projectBasePath, projectName, kf, dim, eps, csv_filename):
    v0 = time.perf_counter()
    
    # Get flaky and non-flaky data
    flakyDir = os.path.join(extractDir, 'flaky')
    nonFlakyDir = os.path.join(extractDir, 'nonFlaky')
    dataPointsFlaky = getDataPoints(flakyDir)
    dataPointsNonFlaky = getDataPoints(nonFlakyDir)
    dataPoints = dataPointsFlaky + dataPointsNonFlaky
    
    print("Data points before vectorization:", len(dataPoints))
    
    # Vectorize data
    Z = vectorization(dataPoints, dim=dim, eps=eps)
    dataPointsList = Z.toarray()  # Ensure it's 2D
    dataLabelsList = np.array([1] * len(dataPointsFlaky) + [0] * len(dataPointsNonFlaky))
    
    # Split data
    for (train_index, test_index) in kf.split(dataPointsList, dataLabelsList):
        trainData, testData = dataPointsList[train_index], dataPointsList[test_index]
        trainLabels, testLabels = dataLabelsList[train_index], dataLabelsList[test_index]

        # Define the parameter grid for GridSearchCV
        param_grid = {
            "n_estimators": [50, 100, 200],
            "criterion": ["gini", "entropy"],
            "max_depth": [10, 30, 50, 100, 300, 500],
            "min_samples_split": [2, 5],
            "min_samples_leaf": [1, 2]
        }

        # Instantiate and perform GridSearchCV
        rf = RandomForestClassifier(random_state=42)
        grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
        grid_search.fit(trainData, trainLabels)

        # Open CSV file for writing results
        with open(csv_filename, mode='w', newline='') as csv_file:
            writer = csv.writer(csv_file)
            writer.writerow(["criterion", "max_depth", "min_samples_split", "min_samples_leaf", "precision", "recall", "accuracy", "f1"])

            # Log results for each hyperparameter combination
            for i, params in enumerate(grid_search.cv_results_['params']):
                print(f"Iteration {i}: Params: {params}")
                
                # Train with best parameters
                trainTime, testTime, predictLabels = classificationRandomForest(trainData, trainLabels, testData, params)
                precision, recall, accuracy, f1 = computeResults(testLabels, predictLabels)

                # Write to CSV
                writer.writerow([params['criterion'], params['max_depth'], params['min_samples_split'], params['min_samples_leaf'], precision, recall, accuracy, f1])

    return grid_search.best_params_

###############################################################################
# Main Execution

if __name__ == "__main__":
    # Parameters setup
    flakyZip = "cleaned_flaky_files.zip"
    nonFlakyZip = "reduced_nonflaky_files.zip"
    extractDir = "extracted"
    outDir = "results_Random_Forest"
    os.makedirs(outDir, exist_ok=True)
    os.makedirs(extractDir, exist_ok=True)

    numSplit = 30
    testSetSize = 0.2
    kf = StratifiedShuffleSplit(n_splits=numSplit, test_size=testSetSize)

    # Define the CSV file for results
    csv_filename = os.path.join(outDir, "params-random-forest.csv")

    # Run Random Forest with Grid Search and save results to CSV
    best_params = randomForestWithGridSearch(outDir, extractDir, "projectName", kf, dim=0, eps=0.3, csv_filename=csv_filename)
    print(f"Best hyperparameters found: {best_params}")
    print("Random Forest analysis completed.")





Data points before vectorization: 94
Fitting 5 folds for each of 144 candidates, totalling 720 fits
Iteration 0: Params: {'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50}
Precision: 0.7272727272727273, Recall: 0.8, Accuracy: 0.7368421052631579, F1 Score: 0.7619047619047619
Iteration 1: Params: {'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Precision: 0.8, Recall: 0.8, Accuracy: 0.7894736842105263, F1 Score: 0.8
Iteration 2: Params: {'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Precision: 0.8, Recall: 0.8, Accuracy: 0.7894736842105263, F1 Score: 0.8
Iteration 3: Params: {'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 50}
Precision: 0.7, Recall: 0.7, Accuracy: 0.6842105263157895, F1 Score: 0.7
Iteration 4: Params: {'criterion': 'gini', 'max_depth': 10, 'min_sam

Precision: 0.8, Recall: 0.8, Accuracy: 0.7894736842105263, F1 Score: 0.8
Iteration 39: Params: {'criterion': 'gini', 'max_depth': 100, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 50}
Precision: 0.7, Recall: 0.7, Accuracy: 0.6842105263157895, F1 Score: 0.7
Iteration 40: Params: {'criterion': 'gini', 'max_depth': 100, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 100}
Precision: 0.7777777777777778, Recall: 0.7, Accuracy: 0.7368421052631579, F1 Score: 0.7368421052631579
Iteration 41: Params: {'criterion': 'gini', 'max_depth': 100, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 200}
Precision: 0.7777777777777778, Recall: 0.7, Accuracy: 0.7368421052631579, F1 Score: 0.7368421052631579
Iteration 42: Params: {'criterion': 'gini', 'max_depth': 100, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 50}
Precision: 0.6, Recall: 0.6, Accuracy: 0.5789473684210527, F1 Score: 0.6
Iteration 43: Params: {'criterion': 'gini', 'max_depth': 

Precision: 0.7777777777777778, Recall: 0.7, Accuracy: 0.7368421052631579, F1 Score: 0.7368421052631579
Iteration 78: Params: {'criterion': 'entropy', 'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 50}
Precision: 0.7272727272727273, Recall: 0.8, Accuracy: 0.7368421052631579, F1 Score: 0.7619047619047619
Iteration 79: Params: {'criterion': 'entropy', 'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 100}
Precision: 0.7, Recall: 0.7, Accuracy: 0.6842105263157895, F1 Score: 0.7
Iteration 80: Params: {'criterion': 'entropy', 'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 200}
Precision: 0.7777777777777778, Recall: 0.7, Accuracy: 0.7368421052631579, F1 Score: 0.7368421052631579
Iteration 81: Params: {'criterion': 'entropy', 'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 50}
Precision: 0.7, Recall: 0.7, Accuracy: 0.6842105263157895, F1 Score: 0.7
Iteration 82: Param

Precision: 0.7777777777777778, Recall: 0.7, Accuracy: 0.7368421052631579, F1 Score: 0.7368421052631579
Iteration 117: Params: {'criterion': 'entropy', 'max_depth': 100, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 50}
Precision: 0.7, Recall: 0.7, Accuracy: 0.6842105263157895, F1 Score: 0.7
Iteration 118: Params: {'criterion': 'entropy', 'max_depth': 100, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 100}
Precision: 0.7777777777777778, Recall: 0.7, Accuracy: 0.7368421052631579, F1 Score: 0.7368421052631579
Iteration 119: Params: {'criterion': 'entropy', 'max_depth': 100, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 200}
Precision: 0.7777777777777778, Recall: 0.7, Accuracy: 0.7368421052631579, F1 Score: 0.7368421052631579
Iteration 120: Params: {'criterion': 'entropy', 'max_depth': 300, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50}
Precision: 0.7777777777777778, Recall: 0.7, Accuracy: 0.7368421052631579, F1 Score: 

Precision: 0.7, Recall: 0.7777777777777778, Accuracy: 0.7368421052631579, F1 Score: 0.7368421052631579
Iteration 11: Params: {'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 200}
Precision: 0.8, Recall: 0.8888888888888888, Accuracy: 0.8421052631578947, F1 Score: 0.8421052631578947
Iteration 12: Params: {'criterion': 'gini', 'max_depth': 30, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50}
Precision: 0.7, Recall: 0.7777777777777778, Accuracy: 0.7368421052631579, F1 Score: 0.7368421052631579
Iteration 13: Params: {'criterion': 'gini', 'max_depth': 30, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Precision: 0.7, Recall: 0.7777777777777778, Accuracy: 0.7368421052631579, F1 Score: 0.7368421052631579
Iteration 14: Params: {'criterion': 'gini', 'max_depth': 30, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Precision: 0.8, Recall: 0.8888888888888888, Accuracy: 0.8421052631578947,

...

Precision: 0.8, Recall: 0.8, Accuracy: 0.7894736842105263, F1 Score: 0.8
Iteration 105: Params: {'criterion': 'entropy', 'max_depth': 50, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 50}
Precision: 0.8181818181818182, Recall: 0.9, Accuracy: 0.8421052631578947, F1 Score: 0.8571428571428571
Iteration 106: Params: {'criterion': 'entropy', 'max_depth': 50, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 100}
Precision: 0.8, Recall: 0.8, Accuracy: 0.7894736842105263, F1 Score: 0.8
Iteration 107: Params: {'criterion': 'entropy', 'max_depth': 50, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 200}
Precision: 0.8, Recall: 0.8, Accuracy: 0.7894736842105263, F1 Score: 0.8
Iteration 108: Params: {'criterion': 'entropy', 'max_depth': 100, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50}
Precision: 0.9, Recall: 0.9, Accuracy: 0.8947368421052632, F1 Score: 0.9
Iteration 109: Params: {'criterion': 'entropy', 'max_depth': 100, 'min_sam

Precision: 0.8, Recall: 0.8, Accuracy: 0.7894736842105263, F1 Score: 0.8
Fitting 5 folds for each of 144 candidates, totalling 720 fits
Iteration 0: Params: {'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50}
Precision: 0.9, Recall: 0.9, Accuracy: 0.8947368421052632, F1 Score: 0.9
Iteration 1: Params: {'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Precision: 0.9, Recall: 0.9, Accuracy: 0.8947368421052632, F1 Score: 0.9
Iteration 2: Params: {'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Precision: 0.9, Recall: 0.9, Accuracy: 0.8947368421052632, F1 Score: 0.9
Iteration 3: Params: {'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 50}
Precision: 0.9, Recall: 0.9, Accuracy: 0.8947368421052632, F1 Score: 0.9
Iteration 4: Params: {'criterion': 'gini', 'max_depth': 10, 'm

Precision: 0.9, Recall: 0.9, Accuracy: 0.8947368421052632, F1 Score: 0.9
Iteration 42: Params: {'criterion': 'gini', 'max_depth': 100, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 50}
Precision: 0.9, Recall: 0.9, Accuracy: 0.8947368421052632, F1 Score: 0.9
Iteration 43: Params: {'criterion': 'gini', 'max_depth': 100, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 100}
Precision: 0.9, Recall: 0.9, Accuracy: 0.8947368421052632, F1 Score: 0.9
Iteration 44: Params: {'criterion': 'gini', 'max_depth': 100, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 200}
Precision: 0.9, Recall: 0.9, Accuracy: 0.8947368421052632, F1 Score: 0.9
Iteration 45: Params: {'criterion': 'gini', 'max_depth': 100, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 50}
Precision: 0.9, Recall: 0.9, Accuracy: 0.8947368421052632, F1 Score: 0.9
Iteration 46: Params: {'criterion': 'gini', 'max_depth': 100, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estim

Precision: 0.8181818181818182, Recall: 0.9, Accuracy: 0.8421052631578947, F1 Score: 0.8571428571428571
Iteration 84: Params: {'criterion': 'entropy', 'max_depth': 30, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50}
Precision: 0.9, Recall: 0.9, Accuracy: 0.8947368421052632, F1 Score: 0.9
Iteration 85: Params: {'criterion': 'entropy', 'max_depth': 30, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Precision: 0.8888888888888888, Recall: 0.8, Accuracy: 0.8421052631578947, F1 Score: 0.8421052631578947
Iteration 86: Params: {'criterion': 'entropy', 'max_depth': 30, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Precision: 0.8181818181818182, Recall: 0.9, Accuracy: 0.8421052631578947, F1 Score: 0.8571428571428571
Iteration 87: Params: {'criterion': 'entropy', 'max_depth': 30, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 50}
Precision: 0.9, Recall: 0.9, Accuracy: 0.8947368421052632, F1 Score: 0.9
Iteration 88: Param

Precision: 0.8181818181818182, Recall: 0.9, Accuracy: 0.8421052631578947, F1 Score: 0.8571428571428571
Iteration 123: Params: {'criterion': 'entropy', 'max_depth': 300, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 50}
Precision: 0.9, Recall: 0.9, Accuracy: 0.8947368421052632, F1 Score: 0.9
Iteration 124: Params: {'criterion': 'entropy', 'max_depth': 300, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 100}
Precision: 0.8888888888888888, Recall: 0.8, Accuracy: 0.8421052631578947, F1 Score: 0.8421052631578947
Iteration 125: Params: {'criterion': 'entropy', 'max_depth': 300, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 200}
Precision: 0.8181818181818182, Recall: 0.9, Accuracy: 0.8421052631578947, F1 Score: 0.8571428571428571
Iteration 126: Params: {'criterion': 'entropy', 'max_depth': 300, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 50}
Precision: 0.9, Recall: 0.9, Accuracy: 0.8947368421052632, F1 Score: 0.9
Iteration 1

In [11]:
import os
import time
import csv
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.random_projection import johnson_lindenstrauss_min_dim, SparseRandomProjection
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
from sklearn.model_selection import StratifiedShuffleSplit, GridSearchCV
import warnings

warnings.filterwarnings("ignore")

###############################################################################
# Helper Functions

def getDataPoints(path):
    """Collects content of all .py files within the given directory."""
    dataPointsList = []
    if not os.path.exists(path):
        print(f"Directory does not exist: {path}")
        return dataPointsList

    for root, dirs, files in os.walk(path):
        for dataPointName in files:
            if dataPointName.endswith(".py"):  # Only consider Python files
                file_path = os.path.join(root, dataPointName)
                with open(file_path, encoding="utf-8") as fileIn:
                    dp = fileIn.read().strip()
                    if dp:  # Ensure the document is not empty
                        dataPointsList.append(dp)
                    else:
                        print(f"Empty file: {file_path}")
    
    if len(dataPointsList) == 0:
        print(f"No valid documents found in directory: {path}")
    
    return dataPointsList

def computeResults(testLabels, predictLabels):
    precision = precision_score(testLabels, predictLabels)
    recall = recall_score(testLabels, predictLabels)
    accuracy = accuracy_score(testLabels, predictLabels)
    f1 = f1_score(testLabels, predictLabels)
    
    return precision, recall, accuracy, f1

###############################################################################
# FLAST

def vectorization(dataPoints, dim=0, eps=0.3):
    countVec = CountVectorizer(stop_words=None)  # Disable stop words filtering
    Z_full = countVec.fit_transform(dataPoints)
    if eps == 0:
        Z = Z_full
    else:
        if dim <= 0:
            dim = johnson_lindenstrauss_min_dim(Z_full.shape[0], eps=eps)
        srp = SparseRandomProjection(n_components=dim)
        Z = srp.fit_transform(Z_full)
    return Z

def classificationDecisionTree(trainData, trainLabels, testData, params):
    # Log the parameters being used for training
    print(f"Training Decision Tree with parameters: {params}")
    
    # Create the DecisionTreeClassifier with the selected hyperparameters
    clf = DecisionTreeClassifier(
        criterion=params.get("criterion", "gini"),
        splitter=params.get("splitter", "best"),
        max_depth=params.get("max_depth"),  # Ensure max_depth is correctly passed
        min_samples_split=params.get("min_samples_split", 2),
        min_samples_leaf=params.get("min_samples_leaf", 1),
    )

    # Fit the classifier
    t0 = time.perf_counter()
    clf.fit(trainData, trainLabels)
    t1 = time.perf_counter()
    trainTime = t1 - t0

    # Make predictions
    t0 = time.perf_counter()
    predictLabels = clf.predict(testData)
    t1 = time.perf_counter()
    testTime = t1 - t0

    return trainTime, testTime, predictLabels

from sklearn.model_selection import train_test_split

def decisionTreeWithGridSearch(outDir, projectBasePath, projectName, dim, eps, csv_filename):
    v0 = time.perf_counter()

    # Load data from directories
    flakyDir = os.path.join(extractDir, 'flaky')
    nonFlakyDir = os.path.join(extractDir, 'nonFlaky')
    dataPointsFlaky = getDataPoints(flakyDir)
    dataPointsNonFlaky = getDataPoints(nonFlakyDir)
    dataPoints = dataPointsFlaky + dataPointsNonFlaky
    print("Data points before vectorization:", len(dataPoints))

    # Vectorize data
    Z = vectorization(dataPoints, dim=dim, eps=eps)
    
    # Flatten the sparse matrix to 2D (samples, features)
    dataPointsList = Z if not hasattr(Z, 'toarray') else Z.toarray()  # Ensuring 2D array
    dataLabelsList = np.array([1] * len(dataPointsFlaky) + [0] * len(dataPointsNonFlaky))
    v1 = time.perf_counter()
    vecTime = v1 - v0

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(dataPointsList, dataLabelsList, test_size=0.3, random_state=42)

    # Define parameter grid for GridSearchCV
    param_grid = {
        "criterion": ["gini", "entropy"],
        "splitter": ["best", "random"],
        "max_depth": [10, 30, 50, 100, 300, 500],
        "min_samples_split": [2, 5],
        "min_samples_leaf": [1, 2]
    }

    # Open CSV file to write results
    with open(csv_filename, mode='w', newline='') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(["criterion", "cvTest", "splitter", "max_depth", "min_samples_split", "min_samples_leaf", "precision", "recall", "accuracy", "f1", "train_time", "test_time"])

        # Instantiate DecisionTreeClassifier
        dt = DecisionTreeClassifier(random_state=42)

        # Perform Grid Search with Cross-Validation
        for cvTest in [3, 5]:
            grid_search = GridSearchCV(estimator=dt, param_grid=param_grid, cv=cvTest, n_jobs=-1, verbose=2)
            grid_search.fit(X_train, y_train)

            # Log results
            for i in range(len(grid_search.cv_results_['params'])):
                params = grid_search.cv_results_['params'][i]
                mean_score = grid_search.cv_results_['mean_test_score'][i]
                mean_fit_time = grid_search.cv_results_['mean_fit_time'][i]
                mean_score_time = grid_search.cv_results_['mean_score_time'][i]

                # Train and test with current parameters
                trainTime, testTime, predictLabels = classificationDecisionTree(X_train, y_train, X_test, params)
                precision, recall, accuracy, f1 = computeResults(y_test, predictLabels)

                # Write results to CSV
                writer.writerow([params['criterion'], cvTest, params['splitter'], params['max_depth'], 
                                 params['min_samples_split'], params['min_samples_leaf'], 
                                 precision, recall, accuracy, f1, mean_fit_time, mean_score_time])

    return grid_search.best_params_



###############################################################################
# Main Execution
if __name__ == "__main__":
    # Parameters setup
    flakyZip = "cleaned_flaky_files.zip"
    nonFlakyZip = "reduced_nonflaky_files.zip"
    extractDir = "extracted"
    outDir = "results_DecisionTree"
    
    # Ensure the directories for output exist
    os.makedirs(outDir, exist_ok=True)
    os.makedirs(extractDir, exist_ok=True)

    # Define the CSV file for results
    csv_filename = os.path.join(outDir, "params-DecisionTree.csv")

    # Run Decision Tree with Grid Search and save results to CSV
    best_params = decisionTreeWithGridSearch(outDir, extractDir, "projectName", dim=0, eps=0.3, csv_filename=csv_filename)
    
    print(f"Best hyperparameters found: {best_params}")
    print("Decision Tree analysis completed.")



Data points before vectorization: 94


PermissionError: [Errno 13] Permission denied: 'results_DecisionTree\\params-DecisionTree.csv'